In [8]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import os
import matplotlib.pyplot as plt
import cv2

In [9]:
img_dims = 64
batch_size = 32

In [10]:
import tensorflow as tf

In [11]:
classifier = tf.keras.Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (img_dims, img_dims, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [12]:
input_path = 'chest_xray/'
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(directory=input_path+'train',
target_size = (img_dims, img_dims),
batch_size = batch_size,
class_mode = 'binary')
test_set = test_datagen.flow_from_directory(directory=input_path+'test',
target_size = (img_dims, img_dims),
batch_size = batch_size,
class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [13]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
epochs = 10

hist = classifier.fit_generator(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, validation_data=test_set, 
           validation_steps= test_set.samples)

c:\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
163/163 [==============================] - 106s 642ms/step - loss: 0.8061 - accuracy: 0.7370 - val_loss: 0.3065 - val_accuracy: 0.8894
Epoch 2/10
163/163 [==============================] - 79s 486ms/step - loss: 0.2340 - accuracy: 0.9046
Epoch 3/10
163/163 [==============================] - 78s 480ms/step - loss: 0.2087 - accuracy: 0.9120
Epoch 4/10
163/163 [==============================] - 79s 484ms/step - loss: 0.2195 - accuracy: 0.9120
Epoch 5/10
163/163 [==============================] - 78s 481ms/step - loss: 0.1734 - accuracy: 0.9362
Epoch 6/10
163/163 [==============================] - 79s 485ms/step - loss: 0.1720 - accuracy: 0.9346
Epoch 7/10
163/163 [==============================] - 80s 493ms/step - loss: 0.1499 - accuracy: 0.9434
Epoch 8/10
163/163 [==============================] - 80s 491ms/step - loss: 0.1585 - accuracy: 0.9334
Epoch 9/10
163/163 [==============================] - 81s 496ms/step - loss: 0.1503 - accuracy: 0.9422
Epoch 10/10
163/163 [=========

In [24]:
img = plt.imread('chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg')
img = cv2.resize(img, (img_dims, img_dims))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = classifier.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] > 0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(prediction)
img=cv2.resize(img,(780,540))
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img,prediction,(10,500), font, 1,(255,255,255),2)
cv2.imshow("img",img)
cv2.waitKey(0)

Pnuemonia


-1

In [30]:
img = plt.imread('chest_xray/test/NORMAL/IM-0019-0001.jpeg')
img = cv2.resize(img, (img_dims, img_dims))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = classifier.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] > 0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(prediction)
img=cv2.resize(img,(780,540))
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img,prediction,(10,500), font, 1,(255,255,255),2)
cv2.imshow("img",img)
cv2.waitKey(0)

Normal


-1

In [20]:
test_data = []
test_labels = []
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
img_dims = 64
input_path = 'chest_xray/'
for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
test_data = np.array(test_data)
test_labels = np.array(test_labels)
from sklearn.metrics import accuracy_score, confusion_matrix

preds = classifier.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)
print('\nTEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((hist.history['accuracy'][-1])*100, 2)))

CONFUSION MATRIX ------------------
[[132 102]
 [  3 387]]

TEST METRICS ----------------------
Accuracy: 83.17307692307693%
Precision: 79.14110429447852%
Recall: 99.23076923076923%
F1-score: 88.05460750853243

TRAIN METRIC ----------------------
Train acc: 94.88
